<a href="https://colab.research.google.com/github/ruthu06/AI-Powered-Chat-Application-for-skincare-Recommendation/blob/main/skincare_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
from datasets import load_dataset

ds = load_dataset("UrFavB0i/skincare-ecommerce-FAQ")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/10.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/5.82k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/104 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/27 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "facebook/blenderbot-400M-distill"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/310k [00:00<?, ?B/s]

In [ ]:
# Preprocessing function
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
import os
os.environ["WANDB_DISABLED"] = "true"

# Split the original dataset into train and validation
train_test_split = ds["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split["train"]
validation_dataset = train_test_split["test"]

# Preprocess function
def preprocess(example):
    inputs = tokenizer(example["Buyer"], max_length=512, truncation=True)
    labels = tokenizer(example["AI Assistant"], max_length=512, truncation=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

# Tokenize train and validation datasets
tokenized_train = train_dataset.map(preprocess, batched=True)
tokenized_validation = validation_dataset.map(preprocess, batched=True)

# Preprocess the test set if required (optional)
tokenized_test = ds["test"].map(preprocess, batched=True)

# Combine the tokenized datasets into a dictionary
tokenized_dataset = {
    "train": tokenized_train,
    "validation": tokenized_validation,
    "test": tokenized_test
}

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./blenderbot-skincare-fine-tuned",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=10
)

# Data collator for padding
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the trainer
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./blenderbot-skincare-fine-tuned")



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-15-080dde86d191>:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,1.727200,1.431979
2,0.677600,1.324900
3,0.443200,1.295558
4,0.291800,1.289297
5,0.161900,1.306398


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 60, 'min_length': 20, 'num_beams': 10, 'length_penalty': 0.65, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [ ]:
# Load the fine-tuned model
from transformers import pipeline

fine_tuned_model = AutoModelForSeq2SeqLM.from_pretrained("/content/blenderbot-skincare-fine-tuned")
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("/content/blenderbot-skincare-fine-tuned")

# Create pipeline for chat
chatbot = pipeline("text2text-generation", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

# Test with a prompt
test_prompt = "You are an expert dermatologist. Based on user input such as following skin type dry skin, concerns of acne, and age 16, create a personalized skincare routine using widely available products and explain why they are suitable"
response = chatbot(test_prompt, max_length=100, num_beams=4)
print("Prompt:", test_prompt)
print("Response:", response[0]["generated_text"])


Prompt: You are an expert dermatologist. Based on user input such as following skin type dry skin, concerns of acne, and age 16, create a personalized skincare routine using widely available products and explain why they are suitable
Response:  Let me check this out for you. You can find a wide variety of skincare products for different ages and levels of skin care. Would you like to know more about these products?


In [ ]:
!pip install transformers huggingface_hub


In [ ]:
from huggingface_hub import login
login()


In [ ]:
model.push_to_hub("skincare")  # Replace "your_model_name" with a name for your model
tokenizer.push_to_hub("skincare")  # Replace "your_model_name" with the same name for the tokenizer


model.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ruthu1/skincare/commit/5f1996f2422a0bb86fcf6b6d4f05534b7bf77eb8', commit_message='Upload tokenizer', commit_description='', oid='5f1996f2422a0bb86fcf6b6d4f05534b7bf77eb8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ruthu1/skincare', endpoint='https://huggingface.co', repo_type='model', repo_id='Ruthu1/skincare'), pr_revision=None, pr_num=None)